<a href="https://colab.research.google.com/github/mknomics/teaching/blob/main/API_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Data from AlphaAdvantage API




## Preliminaries
**Option 1:**


1.   Check WolfDen for a document that has your name and API key.  This is a special key that is better than the free key that you can get from the site.
2.   Make a record of your API key and keep it private for your own use. 

**Option 2:** 


1. [Register for a free API key](https://www.alphavantage.co/support/#api-key) with Alpha Advantage
2. Use your Newberry.edu email for registration
3. Record your key in a safe place


## Read the API Documentation
[Link to API Doccumentation](https://www.alphavantage.co/documentation/)

Alpha Vantage APIs are grouped into five categories: 
1. Time Series Stock APIs
2. Fundamental Data, 
3. Physical and Digital/Crypto Currencies (e.g., Bitcoin), 
4. Economic Indicators, and 
5. Technical Indicators. 

This API uses http protocol, so we structure our queries in what looks like an internet (http) address.  The address is structured using API Parameters according to the syntax in the documentation.  Some parameters are required and others are optional.  

The left menu of the [Documentation](https://www.alphavantage.co/documentation/) shows all of the API Data types that are available at Alpha Advantage.  We will start by extracting Weekly Adjusted Stock data.



In [1]:
import requests 
import pandas as pd
from google.colab import files

def quarter(month=True):
  temp=month/3
  if temp<=1:
    return 1
  elif temp>1 and temp<=2:
    return 2
  elif temp>2 and temp<=3:
    return 3
  else:
    return 4


def Stocks(ticker=True):
  url='https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='+ticker+'&apikey=A601RIXZHX349L2M'
  r = requests.get(url)
  data = r.json()
  stock=pd.DataFrame.from_dict(data['Monthly Adjusted Time Series']).T
  stock.reset_index(inplace=True)
  stock.rename(columns={'index':'date'},inplace=True)
  stock['date']= pd.to_datetime(stock['date'])
  stock['year']=stock['date'].dt.year
  stock['month']=stock['date'].dt.month
  columns=stock.columns
  for i in columns[1:8]:
    stock[i]=stock[i].astype(float)
  stock['quarter']=stock.apply(lambda x: quarter(x['month']),axis=1)
  stock=stock.groupby(['year','quarter'])['1. open','7. dividend amount'].mean().reset_index()
  stock.rename(columns={'1. open':'open','7. dividend amount':'dividend'},inplace=True)
  stock['ticker']=ticker
  return stock


In [ ]:
# Outline for how to concatinate multiple time-series into a single panel

# Toy Example with two dataframes
A=Stocks(ticker='A')
IBM=Stocks(ticker='IBM')
pd.concat([IBM,A])


# General format to concatinate over a list of ticker names:

stock_list=['IBM','A'] #List of ticker names of arbitrary length.  Here the length is = 2
temp=pd.DataFrame() # Initialize empty dataframe

for j in stock_list:  #Loop over the list
  T=Stocks(ticker=j) # Defines a dataframe "T" for ticker "j" on the list
  temp=pd.concat([temp,T]) # Redefines "temp".  Run through how this line works on each iteration of the loop.
temp

In [58]:
# You can run this block line-by-line as a learning tool. 
ticker='IBM'
url='https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='+ticker+'&apikey=A601RIXZHX349L2M'
r = requests.get(url)
data = r.json()
stock=pd.DataFrame.from_dict(data['Monthly Adjusted Time Series']).T
stock.reset_index(inplace=True)
stock.rename(columns={'index':'date'},inplace=True)
stock['date']=pd.to_datetime(stock['date'])
stock.dtypes
stock['month']=stock['date'].dt.month
stock['year']=stock['date'].dt.year
stock.head()
stock['quarter'] = stock.apply(lambda x: quarterly(x['month']),axis=1)
stock.head()
#stock.groupby(['year','quarter'])['1. open'].mean()
columns=stock.columns

Index(['date', '1. open', '2. high', '3. low', '4. close', '5. adjusted close',
       '6. volume', '7. dividend amount', 'month', 'year', 'quarter'],
      dtype='object')

### Earnings Data
Use the API to get Earning Data for IBM

**Always Begin by Looking at API Documentation**
https://www.alphavantage.co/documentation/

In [44]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()
data.keys()

earnings=pd.DataFrame.from_dict(data['quarterlyEarnings'])
earnings.rename(columns={'fiscalDateEnding':'date'},inplace=True)
earnings['date']= pd.to_datetime(earnings['date'])
earnings['month']=earnings['date'].dt.month
earnings['quarter']=earnings['month']/3
earnings.head()

,date,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,month,quarter
0,2022-03-31,2022-04-19,1.4,1.38,0.02,1.4493,3,1.0
1,2021-12-31,2022-01-24,3.35,3.3,0.05,1.5152,12,4.0
2,2021-09-30,2021-10-20,2.52,2.5147,0.0053,0.2108,9,3.0
3,2021-06-30,2021-07-19,2.33,2.2884,0.0416,1.8179,6,2.0
4,2021-03-31,2021-04-19,1.77,1.6524,0.1176,7.1169,3,1.0
